In [ ]:
import osmnx as ox
import pandas as pd
import networkx as nx
import matplotlib as plt

import folium
from folium.plugins import HeatMap

import geopandas as gpd
from shapely.geometry import Point

ox.__version__

In [ ]:
G = ox.graph_from_place("Chicago, Illinois, USA", network_type="all")
filepath = "maps/graph/chicago.graphml"
ox.save_graphml(G, filepath)

fig, ax = ox.plot_graph(G)

In [ ]:
crime_df = pd.read_csv('contextdata/crime/chicago_crimes_2001_2024.csv')

mobility_related_locals = ['STREET', 'RESIDENCE - PORCH / HALLWAY', 'SIDEWALK', 'RESIDENCE PORCH/HALLWAY', 'POLICE FACILITY/VEH PARKING LOT', 'VEHICLE NON-COMMERCIAL', 'DRIVEWAY - RESIDENTIAL', 'PARKING LOT / GARAGE (NON RESIDENTIAL)', 'RESIDENCE-GARAGE', 'GAS STATION', 'PARKING LOT/GARAGE(NON.RESID.)', 'CTA TRAIN', 'CHA PARKING LOT / GROUNDS', 'RESIDENCE - GARAGE', 'ALLEY', 'CTA BUS', 'POLICE FACILITY / VEHICLE PARKING LOT', 'CAR WASH', 'CTA BUS STOP', 'VEHICLE - COMMERCIAL', 'OTHER COMMERCIAL TRANSPORTATION', 'AIRPORT PARKING LOT', 'VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)', 'GAS STATION DRIVE/PROP.', 'TAXICAB', 'CTA PARKING LOT / GARAGE / OTHER PROPERTY', 'CHA PARKING LOT/GROUNDS', 'VEHICLE - DELIVERY TRUCK', 'DELIVERY TRUCK', 'CTA GARAGE / OTHER PROPERTY', 'VEHICLE-COMMERCIAL', 'AUTO', 'HIGHWAY/EXPRESSWAY', 'BRIDGE','VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS', 'GARAGE', 'PARKING LOT', 'GANGWAY', 'VEHICLE - COMMERCIAL: TROLLEY BUS', 'YARD', 'PORCH', 'DRIVEWAY', 'CTA "L" TRAIN',  'BEACH', 'SEWER', 'GARAGE/AUTO REPAIR', 'TRUCK', 'CHA PARKING LOT', 'EXPRESSWAY EMBANKMENT', 'TAXI CAB', 'COACH HOUSE', 'LAGOON', 'TRUCKING TERMINAL', 'VEHICLE - OTHER RIDE SERVICE', 'VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)', 'VEHICLE-COMMERCIAL - TROLLEY BUS', 'VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS']
mobility_crime_df_all_columns = crime_df[crime_df['Location Description'].isin(mobility_related_locals)]

columns_to_keep = ['ID', 'Primary Type', 'Location Description', 'Latitude', 'Longitude']
mobility_crime_df = mobility_crime_df_all_columns[columns_to_keep]

mobility_crime_df.to_csv('contextdata/crime/filtered_mobility_crime_data.csv', index=False)
print("DataFrame saved as 'filtered_crime_data.csv'.")

In [ ]:
#Assuming your DataFrame has 'latitude' and 'longitude' columns
# Drop rows with missing latitude or longitude values
mobility_crime_df = mobility_crime_df.dropna(subset=['Latitude', 'Longitude'])

# Initialize a map centered on the city (you can adjust the location and zoom level)
map_center = [mobility_crime_df['Latitude'].mean(), mobility_crime_df['Longitude'].mean()]
crime_map = folium.Map(location=map_center, zoom_start=12)

# Create a list of latitude and longitude pairs for each crime
crime_locations = mobility_crime_df[['Latitude', 'Longitude']].values.tolist()

# Add these locations as a heat map layer (you can also plot individual markers)
HeatMap(crime_locations).add_to(crime_map)

# Display the map
crime_map.save('crime_distribution_map.html')

print("Map saved as 'crime_distribution_map.html'. Open it in a browser to view.")

In [ ]:
# Get the city graph (replace with the city you're analyzing)
# G = ox.graph_from_place('City Name, Country', network_type='drive')

# Convert the graph nodes to a GeoDataFrame
nodes, edges = ox.graph_to_gdfs(G)

# Load the crime data
#df = pd.read_csv('your_file.csv')

# Assuming the DataFrame has 'latitude' and 'longitude' columns
# Drop rows with missing latitude or longitude values
mobility_crime_df = mobility_crime_df.dropna(subset=['Latitude', 'Longitude'])

# Convert crime locations to a GeoSeries
crime_points = gpd.GeoSeries([Point(xy) for xy in zip(mobility_crime_df['Longitude'], mobility_crime_df['Latitude'])], crs="EPSG:4326")

# Project the graph and crime points to the same coordinate system
crime_points = crime_points.to_crs(nodes.crs)

# Snap each crime point to the nearest edge in the graph
nearest_edges = ox.nearest_edges(G, mobility_crime_df['Longitude'].values, mobility_crime_df['Latitude'].values)

# Add the nearest edge information to the DataFrame
mobility_crime_df['nearest_edge'] = nearest_edges

# Save the updated DataFrame with nearest edge info
mobility_crime_df.to_csv('contextdata/crime/mobility_crime_link_to_edge.csv', index=False)

print("Crime data saved with nearest edges as 'crime_data_with_edges.csv'.")


In [ ]:
# Load the city graph
#G = ox.graph_from_place('City Name, Country', network_type='drive')

# Convert the graph nodes and edges to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(G)

# Load the crime data
#df = pd.read_csv('your_file.csv')

# Assuming the DataFrame has 'latitude' and 'longitude' columns
# Drop rows with missing latitude or longitude values
#mobility_crime_df = mobility_crime_df.dropna(subset=['Latitude', 'Longitude'])

# Snap each crime point to the nearest edge in the graph
#nearest_edges = ox.nearest_edges(G, df['Longitude'].values, df['Latitude'].values)

# Add the nearest edge information to the DataFrame
#mobility_crime_df['nearest_edge'] = nearest_edges

# Aggregate the number of crimes per edge
crime_count_per_edge = mobility_crime_df['nearest_edge'].value_counts().reset_index()
crime_count_per_edge.columns = ['edge', 'crime_count']

# Convert edge tuples to the format used in the graph (u, v, key)
crime_count_per_edge['edge'] = crime_count_per_edge['edge'].apply(lambda x: (x[0], x[1], 0))  # Assuming key=0 for simple graphs

# Update the edge crime count in the graph
for edge, crime_count in crime_count_per_edge[['edge', 'crime_count']].values:
    if G.has_edge(edge[0], edge[1], edge[2]):
        G[edge[0]][edge[1]][edge[2]]['crime_count'] = crime_count
    else:
        # If edge doesn't exist (although it should), set crime count to 0
        G[edge[0]][edge[1]][edge[2]]['crime_count'] = 0

# Save the updated graph
ox.save_graphml(G, filepath='chicago_link_edge_crime.graphml')

print("Graph updated with crime occurrences and saved as 'city_graph_with_crime.graphml'.")


In [ ]:
# Create a color list based on the crime count for each edge
edge_color = [
    data['crime_count'] if 'crime_count' in data else 0 
    for u, v, k, data in G.edges(keys=True, data=True)
]

# Normalize crime counts to a color scale (e.g., Reds)
norm = plt.Normalize(vmin=min(edge_color), vmax=max(edge_color))
sm = plt.cm.ScalarMappable(cmap='Reds', norm=norm)
sm.set_array([])

# Map crime counts to colors
edge_colors = [sm.to_rgba(crime) for crime in edge_color]

# Plot the graph with the crime color scale
fig, ax = ox.plot_graph(
    G, 
    edge_color=edge_colors, 
    edge_linewidth=2, 
    node_size=0
)

# Add colorbar to the plot
plt.colorbar(sm, ax=ax, label="Crime Count")
plt.show()